### Your lab resolution :

In [101]:
import requests
import json
from dotenv import load_dotenv
import os
load_dotenv()
from datetime import datetime, timedelta
import pandas as pd

In [ ]:
json_file_path = 'output.json'

In [80]:
token = os.getenv("token")

In [25]:
# GitHub API endpoint
# api_url = 'https://api.github.com/repos/OWNER/REPO/forks'

owner = 'ironhack-datalabs'
# owner = 'fgardete'
repo = 'bcn-feb-2019'
api_url = f'https://api.github.com/repos/{owner}/{repo}/forks'
# api_url = f'https://api.github.com/repos/{owner}/{repo}/languages'

# Headers for the API request
headers = {
    'Accept': 'application/vnd.github+json',
    'Authorization': f'Bearer {token}',
    'X-GitHub-Api-Version': '2022-11-28',
}

# Make the API request
response = requests.get(api_url, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Successfully retrieved data
    repo_data = response.json()
    print(repo_data)
else:
    # Handle errors
    print(f"Error: {response.status_code}")
    print(response.json())

[{'id': 561512398, 'node_id': 'R_kgDOIXf_zg', 'name': 'bcn-feb-2019', 'full_name': 'fgardete/bcn-feb-2019', 'private': False, 'owner': {'login': 'fgardete', 'id': 87324421, 'node_id': 'MDQ6VXNlcjg3MzI0NDIx', 'avatar_url': 'https://avatars.githubusercontent.com/u/87324421?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/fgardete', 'html_url': 'https://github.com/fgardete', 'followers_url': 'https://api.github.com/users/fgardete/followers', 'following_url': 'https://api.github.com/users/fgardete/following{/other_user}', 'gists_url': 'https://api.github.com/users/fgardete/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/fgardete/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/fgardete/subscriptions', 'organizations_url': 'https://api.github.com/users/fgardete/orgs', 'repos_url': 'https://api.github.com/users/fgardete/repos', 'events_url': 'https://api.github.com/users/fgardete/events{/privacy}', 'received_events_url': 'https://api.git

In [33]:
all_owners = []

for repo in repo_data:
    owner = repo["owner"]['login']

    all_owners.append(owner)


In [34]:
all_owners

['fgardete',
 'StanRocha',
 'MRibas123',
 'MariaEstebanDA',
 'je-nakajima',
 'franbaldi',
 'brunacarolino']

In [55]:
def get_languages(owners):
    unique_languages= set()
    for owner in owners:
        repo = 'bcn-feb-2019'
        api_url = f'https://api.github.com/repos/{owner}/{repo}/languages'


        headers = {
            'Accept': 'application/vnd.github+json',
            'Authorization': f'Bearer {token}',
            'X-GitHub-Api-Version': '2022-11-28',
            }
        response = requests.get(api_url, headers=headers)

        if response.status_code == 200:
            repo_data = response.json()
            for language in repo_data.keys():
                unique_languages.add(language)
                
        else:
  
            print(f"Error: {response.status_code}")
            print(response.json())

    return unique_languages


In [56]:
get_languages(all_owners)

{'HTML', 'Jupyter Notebook', 'Python'}

### Challenge 2: Count Commits

I'm not sure that i understood task correctly, because if i use API for commits i receive only 3 commits from 2022.

In [97]:

owner = 'ta-data-pt-rmt'

repo = 'lab-mysql-first-queries'

api_url = f'https://api.github.com/repos/{owner}/{repo}/commits'


headers = {
    'Accept': 'application/vnd.github+json',
    'Authorization': f'Bearer {token}',
    'X-GitHub-Api-Version': '2022-11-28',
}

# params = {
#     'since': datetime.now() - timedelta(weeks=3),
#      }

# Make the API request with time delta:
# response = requests.get(api_url, params = params, headers=headers)


response = requests.get(api_url, headers=headers)

if response.status_code == 200:
   
    repo_data = response.json()

    all_commits = []

    for commit in repo_data:
        author_info = commit.get('author')
        commit_info = commit.get('commit')

        if author_info and commit_info:
            commit_data = {
                'author_id': author_info.get('id'),
                'date': commit_info['author']['date']
            }
            all_commits.append(commit_data)

    print(all_commits)

else:
  
    print(f"Error: {response.status_code}")
    print(response.json())

[{'author_id': 99891270, 'date': '2022-03-31T18:05:34Z'}, {'author_id': 99891270, 'date': '2022-03-31T18:04:49Z'}, {'author_id': 99891270, 'date': '2022-03-19T11:41:30Z'}]


I thought maybe i need to use pull requests and get SHA for next end point...but also not sure, so an example below i wanted to take all pull requests and filter them on a date. I will be glad if you could clarify for me what end point should be here.

In [104]:
three_weeks_ago = datetime.now() - timedelta(weeks=3)

owner = 'ta-data-pt-rmt'

repo = 'lab-mysql-first-queries'

api_url = f'https://api.github.com/repos/{owner}/{repo}/pulls'
page = 1

headers = {
    'Accept': 'application/vnd.github+json',
    'Authorization': f'Bearer {token}',
    'X-GitHub-Api-Version': '2022-11-28',
}

params = {
    'page': page,
    'per_page':'100',
   }

response = requests.get(api_url,params = params, headers=headers)


if response.status_code == 200:
   
    repo_data = response.json()
    all_pulls = []

    for pull in repo_data:
                author_info = pull.get('user')
                author_name = pull.get('head')
                pull_info = pull.get('head')
                date = pull.get('base')

                if author_info and pull_info and date:
                    date_pushed = date['repo']['pushed_at']
                    date_pushed_dt = datetime.strptime(date_pushed, '%Y-%m-%dT%H:%M:%SZ')

                    if date_pushed_dt >= three_weeks_ago:
                        pull_data = {
                            'author_id': author_info.get('id'),
                            'author_name': author_name.get("label"),
                            'sha': pull_info['sha'],
                            'date_pushed': date_pushed
                        }
                        all_pulls.append(pull_data)
    page += 1
    print (all_pulls)
    print(len(all_pulls))
    
else:
    
    print(f"Error: {response.status_code}")
    print(response.json())

[{'author_id': 166904598, 'author_name': 'alexg119:main', 'sha': '6d34bed7f035a27bb70914568b38d9c5837a9a22', 'date_pushed': '2024-06-13T15:42:46Z'}, {'author_id': 81372197, 'author_name': 'Blyatman-coder:main', 'sha': 'cbab6fa5c3eb5e00bd77fa57192a9c1de6c0b7c9', 'date_pushed': '2024-06-13T15:42:46Z'}, {'author_id': 71112777, 'author_name': 'MontaseerAlam:main', 'sha': '6881be594d37c2eb4d4b7ad20261f1e508fd9e1f', 'date_pushed': '2024-06-13T15:42:46Z'}, {'author_id': 166298445, 'author_name': 'Enterausername357:main', 'sha': 'e4ae75ae6f4f6ac842f6638ce32301587eef6077', 'date_pushed': '2024-06-13T15:42:46Z'}, {'author_id': 165660345, 'author_name': 'hilu1:main', 'sha': '1baa376d9e5b27eb169b8fa25fc38331683273bb', 'date_pushed': '2024-06-13T15:42:46Z'}, {'author_id': 166646497, 'author_name': 'gop997:main', 'sha': '38a0a0c1b2275a4080fbb87b505e892e7d2d404c', 'date_pushed': '2024-06-13T15:42:46Z'}, {'author_id': 134236963, 'author_name': 'Annasnizhko:main', 'sha': '6a58d37286fedb5903deb82350353c